In [25]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import os
import requests
import dateutil
from datetime import datetime

pd.set_option("display.max_rows", 1000)

In [62]:
#https://stackoverflow.com/questions/16627227/http-error-403-in-python-3-web-scraping
requested_page = Request('https://fussball.wettpoint.com/en/tables.html', headers={'User-Agent': 'Chrome/64.0'})
webpage = urlopen(requested_page).read()
overview_page = BeautifulSoup(webpage, "html.parser")

In [64]:
league_list = []
league_club_list = []

for table in overview_page.findAll('table', {'summary':'Soccer Tables'}):
    for td in table.findAll('td', {'class': 'sps2'}):
        for td2 in table.findAll('a', href=True):
            league_dict = {}
            league_name = td2.string
            league_dict['league_name'] = league_name
            print("Got up until this this league:",td2.string)
            url_league_page = td2['href']
            league_dict['league_page_url'] = url_league_page
            league_list.append(league_dict)
            
            #open the page of each league
            search_for = league_name + " Table 2014"
            requested_page_temp = Request(str(url_league_page), headers={'User-Agent': 'Chrome/64.0'})
            webpage_temp = urlopen(requested_page_temp).read()
            overview_page_temp = BeautifulSoup(webpage_temp, "html.parser")

            #searching the page of each league for the 2014 table
            for a in overview_page_temp.findAll('a', href = True):
                if a.string == str(search_for):
                    archive_url = a['href']

                    #accessing the 2014 page of each league
                    requested_page_temp2 = Request(str(archive_url), headers={'User-Agent': 'Chrome/64.0'})
                    webpage_temp2 = urlopen(requested_page_temp2).read()
                    overview_page_temp2 = BeautifulSoup(webpage_temp2, "html.parser")

                    #and scraping all club names from this page
                    for table in overview_page_temp2.findAll('table'):
                        for tds in table.findAll('td', {'class':'content-normal'}):
                            for links in tds.findAll('a', href=True):
                                archive_dict = {}
                                if 'teams' in links['href']:
                                    print(links.string)
                                    archive_dict['league_name'] = league_name
                                    archive_dict['club_name']=links.string
                                    league_club_list.append(archive_dict)

Got up until this this league: Kategoria Superiore
Skënderbeu Korçë
FK Kukësi
KF Tirana
KF Partizani Tirana
KF Laçi
Flamurtari Vlorë
Vllaznia Shkodër
Teuta Durrës
Apolonia Fier
KF Elbasani
Got up until this this league: Ligue 1
MO Béjaïa
ES Sétif
MC Oran
USM Alger
ASM Oran
CR Belouizdad
USM El Harrach
RC Arbaâ
JS Kabylie
JS Saoura
MC Alger
CS Constantine
USM Bel Abbès
ASO Chlef
MC El Eulma
NA Hussein Dey
Got up until this this league: Argentino A - Group A
Brown de Trelew
C.A.I.
Deportivo Madryn
Deportivo Roca
Independiente Neuquén
Cipolletti
Belgrano Santa Rosa
Alianza Cutral Có
Got up until this this league: Argentino A - Group B
Estudiantes San Luis
San Lorenzo Alem
Unión Villa Krause
Juventud U San Luis
Gimnasia de Mendoza
Deportivo Maipú
Américo Tesorieri
Andino La Rioja
Got up until this this league: Argentino A - Group C
Central Córdoba SE
Unión Aconquija
Atlético Mitre
Juventud Antoniana
San Martín Tucumán
Altos Hornos
San Jorge Tucumán
Gimnasia y Tiro
Got up until this this le

In [65]:
league_df = pd.DataFrame(league_list)
league_df.head(3)

,league_name,league_page_url
0,Kategoria Superiore,https://fussball.wettpoint.com/en/tables/alban...
1,Ligue 1,https://fussball.wettpoint.com/en/tables/alger...
2,Argentino A - Group A,https://fussball.wettpoint.com/en/tables/argen...


In [66]:
league_df.tail(3)

,league_name,league_page_url
572,Primera Division Adecuacion,https://fussball.wettpoint.com/en/tables/venez...
573,V-League,https://fussball.wettpoint.com/en/tables/vietn...
574,League Of Wales,https://fussball.wettpoint.com/en/tables/wales...


In [74]:
league_df.to_csv('data/output/world_leagues_overview-20180507.csv')

In [68]:
club_league_df = pd.DataFrame(league_club_list)

In [69]:
club_league_df.head()

,club_name,league_name
0,Skënderbeu Korçë,Kategoria Superiore
1,FK Kukësi,Kategoria Superiore
2,KF Tirana,Kategoria Superiore
3,KF Partizani Tirana,Kategoria Superiore
4,KF Laçi,Kategoria Superiore


In [70]:
club_league_df.tail()

,club_name,league_name
4740,Rhyl FC,League Of Wales
4741,Cefn Druids,League Of Wales
4742,Connah`s Quay,League Of Wales
4743,Bangor City,League Of Wales
4744,Prestatyn Town,League Of Wales


In [73]:
club_league_df.to_csv('data/output/clubs_leagues_2014-20180507.csv')